In [1]:
from sparse_causal_model_learner_rl.sacred_gin_tune.sacred_wrapper import load_config_files
from sparse_causal_model_learner_rl.sacred_gin_tune.tune_to_gin import tune_grid_search
from sparse_causal_model_learner_rl.sacred_gin_tune.test_load_gin_file import f, g


import gin
import tempfile
import pytest
import os
from ray import tune

In [17]:
def g_conf_grid_search():
    conf_file = """
        import sparse_causal_model_learner_rl.sacred_gin_tune.test_load_gin_file
        import sparse_causal_model_learner_rl.sacred_gin_tune.tune_to_gin
        
        ABA = 'caba'
        
        scope1/test_load_gin_file.f.param = 123
        scope2/test_load_gin_file.f.param = 456
        
        tune1/tune_grid_search.values = [@scope1/test_load_gin_file.f(), @scope2/test_load_gin_file.f()]
        tune2/tune_grid_search.values = [%ABA, @test_load_gin_file.f]
        
        test_load_gin_file.g.param1 = @tune1/tune_grid_search()
        test_load_gin_file.g.param2 = @tune2/tune_grid_search()
        """
    return conf_file


In [18]:
with tempfile.NamedTemporaryFile(mode='w+', delete=False) as tmp_config_file:
    tmp_config = tmp_config_file.name
    tmp_config_file.write(g_conf_grid_search())

load_config_files([tmp_config])

['tmp2dr3']

In [4]:
module_fcn_name = tune_grid_search.__module__ + "." + tune_grid_search.__name__
fcn_name = tune_grid_search.__name__

In [70]:
search = {}
for (scope, fcn), args in gin.config._CONFIG.items():
    if fcn == module_fcn_name:
        search[f"{scope}/{fcn_name}"] = tune.grid_search(args['values'])

In [73]:
def fcn(config):
    gin.parse_config(config['gin_config'])

    
    for key, value in config.items():
        if key.endswith('grid_search'):
            scope, name = key.split('/')
            gin.bind_parameter(scope + '/' + name + '.override', value)
            print("bind", key, value, scope, name)
    
    
    res = g()
    print(res)
    return res

In [74]:
#search['num_workers'] = 0
search['gin_config'] = gin.config_str()
res = tune.run(fcn, config=search)

Trial name,status,loc,tune1/tune_grid_search,tune2/tune_grid_search
fcn_39b12_00000,RUNNING,,,
fcn_39b12_00001,PENDING,,,
fcn_39b12_00002,PENDING,,,
fcn_39b12_00003,PENDING,,,


(pid=22407) bind tune1/tune_grid_search 123 tune1 tune_grid_search
(pid=22407) bind tune2/tune_grid_search caba tune2 tune_grid_search
(pid=22407) (123, 'caba')
(pid=22405) bind tune1/tune_grid_search 456 tune1 tune_grid_search
(pid=22405) bind tune2/tune_grid_search caba tune2 tune_grid_search
(pid=22405) (456, 'caba')
(pid=22418) bind tune1/tune_grid_search 123 tune1 tune_grid_search
(pid=22418) bind tune2/tune_grid_search <function f at 0x7fca42a399d8> tune2 tune_grid_search
(pid=22418) (123, <function f at 0x7fca42a399d8>)


Trial name,status,loc,tune1/tune_grid_search,tune2/tune_grid_search
fcn_39b12_00000,TERMINATED,,,
fcn_39b12_00001,TERMINATED,,,
fcn_39b12_00002,TERMINATED,,,
fcn_39b12_00003,TERMINATED,,,


(pid=22527) bind tune1/tune_grid_search 456 tune1 tune_grid_search
(pid=22527) bind tune2/tune_grid_search <function f at 0x7f2e00ad49d8> tune2 tune_grid_search
(pid=22527) (456, <function f at 0x7f2e00ad49d8>)


In [51]:
dir(res)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_checkpoints',
 '_configs',
 '_experiment_dir',
 '_experiment_state',
 '_get_trial_paths',
 '_retrieve_rows',
 '_trial_dataframes',
 '_validate_metric',
 '_validate_mode',
 'best_checkpoint',
 'best_config',
 'best_dataframe',
 'best_logdir',
 'best_result',
 'best_result_df',
 'best_trial',
 'dataframe',
 'default_metric',
 'default_mode',
 'fetch_trial_dataframes',
 'get_all_configs',
 'get_best_checkpoint',
 'get_best_config',
 'get_best_logdir',
 'get_best_trial',
 'get_trial_checkpoints_paths',
 'results',
 'results_df',
 'runner_data',
 'stats',
 'trial_dataframes',
 'trials']

In [47]:
res.dataframe()

ValueError: No `metric` has been passed and  `default_metric` has not been set. Please specify the `metric` parameter.

In [ ]:
()